# Lorentz transformation
In this lesson we will write and use functions for working with 4-vectors and Lorentz transformations.

## Pre-requisites
First we need to load the modules we need. We can also check whether loading was succeful and the version of the imported module.

In [1]:
import astropy
astropy.__version__

'4.2.1'

numpy is typically imported as np:

In [2]:
import numpy as np
np.__version__

'1.20.1'

We will use matplotlib for plotting:

In [3]:
import matplotlib
matplotlib.__version__

'3.3.4'

From the astropy module we need only the units and the constants, we will import them as u and c.

In [4]:
from astropy import units as u
from astropy import constants as c

We will now import the Abstract Base Class (abc) module, which will allow us to define a general 4-vector class followed by 4-momentum and 4-position subclasses.

In [ ]:
import abc

Let's import both the math module and the complex math module.

In [5]:
import cmath
import math

## 4-vectors

Let's first define the Minkowski metric. We will take the time component as the 0th component with positive normalisation.

In [6]:
METRIC = np.diag([0, 0, 0, 0])  # This is a diagonal 4x4 matrix. Replace the 0s with the correct elements along the diagonal.

Let us now define the 4-vector class. The first method describes how to initialize the 4-vector.

In [ ]:
class Vector4:
    """
    Representation of a Lorentz 4-vector.
    """
    def __init__(self, *x):
        """
        Creates a new 4-vector. The arguments can be either the four
        components or another 4-vector.

        >>> Vector4(60, 2, 3, 4)
        Vector4(60, 2, 3, 4)

        >>> vec = Vector4(60, 1, 2, 3)
        >>> Vector4(vec)
        Vector4(60, 1, 2, 3)
        """
        if len(x) == 1 and isinstance(x[0], Vector4):
            x = x[0]
            self._values = np.array(x.components)
        elif len(x) == 4:
            self._values = np.array(list(x))
        else:
            raise TypeError("4-vectors expects four values")

The next two methods define ways to display and to access the components of the 4-vector.

In [ ]:
    def __repr__(self):
        """
        Returns a string representation of the object.
        """
        if self._values.ndim == 1:
            pattern = "%g"
        else:
            pattern = "%r"

        return "%s(%s)" % (self.__class__.__name__,
                           ", ".join([pattern % _ for _ in self._values]))

    @property
    def components(self):
        """
        Returns the interal array of all components.
        """
        return self._values

Definitions of mathematical operations.

In [ ]:
    def __add__(self, other):
        """
        Addition of two 4-vectors. Can only add vectors of the same type (or
        subtype).

        >>> a = Vector4(1, 2, 3, 4)
        >>> b = Vector4(2, 4, 8, 16)
        >>> a + b
        Vector4(3, 6, 11, 20)
        """
        vector = self.__class__(self)
        vector += other
        return vector

    def __iadd__(self, other):
        """
        In-place addition of two 4-vectors.

        >>> a = Vector4(1, 2, 3, 4)
        >>> b = Vector4(2, 4, 8, 16)
        >>> b += a
        >>> b
        Vector4(3, 6, 11, 20)
        """
        self._values = self.components + Vector4(*other).components
        return self

    def __sub__(self, other):
        """
        Subtraction of two 4-vectors.

        >>> a = Vector4(1, 2, 3, 4)
        >>> a - b
        Vector4(-3, -1, 1, 3)
        """
        vector = self.__class__(self)
        vector -= other
        return vector

    def __isub__(self, other):
        """
        In-place subtraction of two 4-vectors.

        >>> a = Vector4(1, 2, 3, 4)
        >>> b = Vector4(4, 3, 2, 1)
        >>> b -= a
        >>> b
        Vector4(-3, -1, 1, 3)
        """
        self._values = self.components - Vector4(*other).components
        return self

    def __mul__(self, other):
        """
        Multiplication of a 4-vector by a scalar or the dot product with
        another 4-vector.
        """
        if hasattr(other, "__len__") and len(other) == 4:
            # Dot product
            other = other.components
            components = self.components

            is_scalar = (other.ndim == 1) and (components.ndim == 1)

            if other.ndim == 1:
                other = other.reshape(other.shape[0], 1)
            if components.ndim == 1:
                components = components.reshape(components.shape[0], 1)

            dot_product = (METRIC.dot(other) * components).sum(axis=0)
            dot_product = dot_product.reshape(dot_product.size)

            if is_scalar:
                return dot_product[0]

            return dot_product

        vector = self.__class__(self)
        vector *= other
        return vector


    def __rmul__(self, other):
        """
        Multiplication of a 4-vector by a scalar from the left.
        """
        return self * other

    def __imul__(self, other):
        """
        Multiplication of a 4-vector by a scalar from the left.
        """
        if hasattr(other, "__len__") and len(other) != 1:
            raise TypeError("In-place multiplication only possible for "
                            "scalars.")

        self._values *= other
        return self

    def __neg__(self):
        """
        Negate all components, equivalent to v * (-1)
        """
        return (-1) * self

    def __truediv__(self, other):
        """
        Division of a 4-vector by a scalar from the left.
        """
        vector = self.__class__(self)
        vector /= other
        return vector

    def __floordiv__(self, other):
        """
        Division of a 4-vector by a scalar from the left.
        """
        vector = self.__class__(self)
        vector //= other
        return vector

    def __ifloordiv__(self, other):
        """
        In-place division of a 4-vector by a scalar.
        """
        if hasattr(other, "__len__") and len(other) != 1:
            raise TypeError("Division only possible by scalars.")

        # in-place div no always possible for numpy arrays depending on their
        # type, i.e. cannot in-place convert int array to float
        self._values = self._values // other
        return self

    def __itruediv__(self, other):
        """
        In-place division of a 4-vector by a scalar.
        """
        if hasattr(other, "__len__") and len(other) != 1:
            raise TypeError("Division only possible by scalars.")

        # in-place div no always possible for numpy arrays depending on their
        # type, i.e. cannot in-place convert int array to float
        self._values = self._values / other
        return self

    def __div__(self, other):
        """
        Legacy support
        """
        return self.__truediv__(other)

    def __idiv__(self, other):
        """
        Legacy support
        """
        return self.__itruediv__(other)


### Using numpy arrays and quantities
Imagine you want to calculate the energies corresponding to many wavelengths. You can use arrays for that. Note that you cannot combine values with different units, even if they have the same dimension.

## Particle Distributions (photons, electrons, protons)
In this course we will not discuss photons of a single energy, but particle distributions of photons, electrons or protons. We will study the particles over several orders of magnitude in energy, let's say from 1 MeV (10⁶ eV) to 100 TeV (10¹⁴ eV). We will create a log space with 15 data points:

We study non-thermal emission, so our photons will not follow a block-body spectrum but typically a power law. A power law has the following form:
$$ F(E) = A \times \left( \frac{E}{E_0} \right)^{-\Gamma} $$
$A$ is the amplitude, i.e. the function's value at the reference energy $E_0$. $\Gamma$ is called the spectral index. Note the minus sign in the exponent, so $\Gamma$ is typically positive. We often use 1 TeV as reference energy:

Let's define some example parameters:

A power law is already defined in astropy. Let's make use of it.

Now we can call this function on our energy array defined above:

### Plotting
We will use pyplot from matplotlib to do some plotting. pyplot is usually imported as plt:

Let's make a plot of our y value over our energy array:

This looks a bit strange. Keep in mind that our energies span 14 orders of magnitude. Let's try to get the energy axis in log scale:

Looks better already. But is our function really zero above an energy of 10^7 eV? Let's try a log-log plot:

Looks good. We see that a power law is represented as a straight line in a log-log plot. Now let's add titles to the axes and a legend:

You see that the y axis is in units TeV⁻¹. It is a differential particle number, how many particles are found in an infinitesimal small bin of with $dE$:
$$\frac{dN}{dE}(E).$$
If you want to know how many particles are in the entire particle spectrum then you have to integrate over it:
$$ N = \int_{E_1}^{E_2} \frac{dN}{dE}(E) dE.$$
For a power law it is easy to do the integration analytically. But it may be difficult for more complicated spectra. We can use numpy for a numerical integration.

There are no units as this is simply a particle number. Keep in mind that the trapezoidal rule is not very accurate and depends on the number of points. Having more points would increase the accuracy. Other, more sophisticated integration modules exist for python.

Similarly we can calculate the total energy in the spectrum. We need to integrate over all energies:
$$ E_{tot} = \int_{E_1}^{E_2} E \frac{dN}{dE}(E) dE.$$
We can achieve this by multiplying the $y$ array with the energies array and then integrate.

### Exercise
Make a plot of three power laws with indices 1.5, 2.0 and 2.5. Keep the amplitude as above. Do not forget the title on the x-axis and the legend. Calculate the number of particles and the total energy in each spectrum.

**[3 marks]**

## Cyclotron Frequency

Here are some values fo an example. We will use the geo-magnetic field:

The electron has one elementary charge, we do not care about the sign here:

Let's put everything together:

Depending on the system you are using the elementary charge has different values and units. So we have to be specific here. All calculations in this course are in standard units, so let's try this.

How many cycles per second? We have to devide by the angle of one full circle, $2\pi$.

### Exercise

We will need the cyclotron frequency more often. Please write a function which takes the magnetic field as first parameter, the "atomic number" of the particle (-1 for an electron, +1 for a proton, +2 for a Helium nucleus and so on) as 2nd parameter with -1 as default, and the mass as third parameter with the electron mass as default. Return the cyclotron frequency in Hz. Make sure that it is never negative!

**[3 marks]**

Let's test it:

## Submission

Before you submit your work you should make a few checks that everything works fine.

1. Save your notebook as a PDF (File->Download As->PDF). This document will help you debugging in the next step.
1. If PDF export does not work: You can do File->Print Preview and then print to a file.
1. Restart the kernel and rerun the entire notebook (Kernel->Restart & Run All). This will delete all variables (but not your code) and rerun the notebook in one go. If this does not go through the endthen you have to fix it. You will see at which cell the run stopped. A common mistake is using a variable that is defined only at a later stage.
1. You think you fixed everything? Redo step 2 (Kernel->Restart & Run All)

You have to download and submit 2 files, the jupyter notebook and a pdf.
- Jupyter notebook. File->Download As->Notebook (.ipynb). Save this file on your disk.
- PDF file. File->Download As->PDF. Save this file on your disk.
- If PDF export does not work. You can do File->Print Preview and then print to a file.

Please submit the two files on Ulwazi.